In [1]:
import pandas as pd

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [4]:
data = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
image_path = '/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv'


In [5]:
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [6]:
image_names = data['image_id'].values
labels = data['dx']

In [7]:
image_names

array(['ISIC_0027419', 'ISIC_0025030', 'ISIC_0026769', ...,
       'ISIC_0033536', 'ISIC_0032854', 'ISIC_0032258'], dtype=object)

In [8]:
labels

0          bkl
1          bkl
2          bkl
3          bkl
4          bkl
         ...  
10010    akiec
10011    akiec
10012    akiec
10013    akiec
10014      mel
Name: dx, Length: 10015, dtype: object

In [9]:
df = pd.read_csv(image_path)
images = df.drop(['label'],axis=1)/255
images.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,0.752941,0.600000,0.756863,0.764706,0.607843,0.752941,0.772549,0.603922,0.725490,0.792157,...,0.525490,0.678431,0.486275,0.541176,0.717647,0.576471,0.650980,0.725490,0.603922,0.694118
1,0.098039,0.054902,0.117647,0.266667,0.188235,0.294118,0.482353,0.364706,0.494118,0.619608,...,0.321569,0.235294,0.152941,0.215686,0.098039,0.054902,0.109804,0.098039,0.054902,0.105882
2,0.752941,0.541176,0.600000,0.784314,0.568627,0.639216,0.788235,0.556863,0.627451,0.807843,...,0.584314,0.654902,0.505882,0.560784,0.623529,0.486275,0.556863,0.533333,0.407843,0.458824
3,0.149020,0.074510,0.117647,0.372549,0.231373,0.282353,0.560784,0.403922,0.466667,0.670588,...,0.286275,0.172549,0.101961,0.141176,0.098039,0.047059,0.066667,0.098039,0.047059,0.058824
4,0.619608,0.443137,0.545098,0.760784,0.564706,0.682353,0.843137,0.635294,0.749020,0.882353,...,0.788235,0.819608,0.650980,0.725490,0.674510,0.529412,0.584314,0.427451,0.305882,0.360784


In [10]:
images 

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,0.752941,0.600000,0.756863,0.764706,0.607843,0.752941,0.772549,0.603922,0.725490,0.792157,...,0.525490,0.678431,0.486275,0.541176,0.717647,0.576471,0.650980,0.725490,0.603922,0.694118
1,0.098039,0.054902,0.117647,0.266667,0.188235,0.294118,0.482353,0.364706,0.494118,0.619608,...,0.321569,0.235294,0.152941,0.215686,0.098039,0.054902,0.109804,0.098039,0.054902,0.105882
2,0.752941,0.541176,0.600000,0.784314,0.568627,0.639216,0.788235,0.556863,0.627451,0.807843,...,0.584314,0.654902,0.505882,0.560784,0.623529,0.486275,0.556863,0.533333,0.407843,0.458824
3,0.149020,0.074510,0.117647,0.372549,0.231373,0.282353,0.560784,0.403922,0.466667,0.670588,...,0.286275,0.172549,0.101961,0.141176,0.098039,0.047059,0.066667,0.098039,0.047059,0.058824
4,0.619608,0.443137,0.545098,0.760784,0.564706,0.682353,0.843137,0.635294,0.749020,0.882353,...,0.788235,0.819608,0.650980,0.725490,0.674510,0.529412,0.584314,0.427451,0.305882,0.360784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,0.717647,0.647059,0.709804,0.713725,0.647059,0.705882,0.721569,0.650980,0.713725,0.737255,...,0.709804,0.815686,0.725490,0.733333,0.815686,0.729412,0.729412,0.807843,0.733333,0.741176
10011,0.007843,0.011765,0.003922,0.149020,0.129412,0.125490,0.474510,0.407843,0.403922,0.517647,...,0.505882,0.376471,0.309804,0.298039,0.094118,0.090196,0.082353,0.011765,0.015686,0.003922
10012,0.517647,0.462745,0.462745,0.654902,0.584314,0.584314,0.686275,0.611765,0.627451,0.721569,...,0.745098,0.800000,0.709804,0.698039,0.709804,0.623529,0.600000,0.674510,0.592157,0.568627
10013,0.627451,0.486275,0.572549,0.643137,0.513725,0.596078,0.654902,0.498039,0.572549,0.662745,...,0.647059,0.725490,0.635294,0.654902,0.721569,0.615686,0.650980,0.725490,0.635294,0.674510


In [11]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

In [12]:
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [13]:
X_train, X_test,y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=42)

In [14]:
X_train.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
6582,0.898039,0.580392,0.654902,0.905882,0.576471,0.658824,0.905882,0.572549,0.666667,0.905882,...,0.600000,0.854902,0.572549,0.596078,0.850980,0.580392,0.596078,0.843137,0.576471,0.600000
7194,0.623529,0.411765,0.560784,0.623529,0.392157,0.537255,0.619608,0.380392,0.513725,0.631373,...,0.733333,0.713725,0.572549,0.701961,0.701961,0.525490,0.666667,0.674510,0.462745,0.600000
9446,0.039216,0.019608,0.031373,0.219608,0.137255,0.176471,0.415686,0.290196,0.352941,0.517647,...,0.517647,0.521569,0.407843,0.482353,0.435294,0.325490,0.396078,0.286275,0.192157,0.250980
533,0.588235,0.537255,0.517647,0.619608,0.584314,0.600000,0.647059,0.611765,0.639216,0.654902,...,0.462745,0.537255,0.482353,0.501961,0.494118,0.431373,0.419608,0.470588,0.411765,0.384314
132,0.513725,0.466667,0.549020,0.533333,0.486275,0.549020,0.549020,0.501961,0.572549,0.564706,...,0.603922,0.596078,0.560784,0.611765,0.576471,0.533333,0.576471,0.560784,0.513725,0.556863


In [15]:
X_train.shape

(8012, 2352)

In [16]:
X_train = X_train.values.reshape(int(len(X_train)),28,28,3)
X_test = X_test.values.reshape(int(len(X_test)),28,28,3)

In [17]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2] * X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2] * X_test.shape[3])


In [18]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(units=32))
model.add(Dropout(0.5))

model.add(Dense(units=labels.shape[1], activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train,y_train, batch_size=32, epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
251/251 [==============================] - 22s 67ms/step - loss: 1.2002 - accuracy: 0.6629 - val_loss: 1.0542 - val_accuracy: 0.6680
Epoch 2/50
251/251 [==============================] - 16s 66ms/step - loss: 1.1050 - accuracy: 0.6692 - val_loss: 1.0168 - val_accuracy: 0.6680
Epoch 3/50
251/251 [==============================] - 16s 63ms/step - loss: 1.0857 - accuracy: 0.6697 - val_loss: 1.0383 - val_accuracy: 0.6680
Epoch 4/50
251/251 [==============================] - 16s 65ms/step - loss: 1.0365 - accuracy: 0.6699 - val_loss: 0.9859 - val_accuracy: 0.6680
Epoch 5/50
251/251 [==============================] - 16s 65ms/step - loss: 1.0151 - accuracy: 0.6697 - val_loss: 1.0370 - val_accuracy: 0.6680
Epoch 6/50
251/251 [==============================] - 16s 64ms/step - loss: 0.9965 - accuracy: 0.6677 - val_loss: 0.9628 - val_accuracy: 0.6680
Epoch 7/50
251/251 [==============================] - 16s 63ms/step - loss: 0.9497 - accuracy: 0.6696 - val_loss: 0.9098 - val_accuracy:

In [21]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f%%' % (accuracy * 100))

63/63 [==============================] - 1s 20ms/step - loss: 0.7850 - accuracy: 0.7264
Accuracy: 72.64%
